In [2]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

base_path = r"E:\project\20210928_loan\data"
train_bank = pd.read_csv(base_path + r'\train_public.csv')
train_internet = pd.read_csv(base_path + r'\train_internet.csv')

test = pd.read_csv(base_path+ r'\test_public.csv')

In [10]:
def print_df(df):
    print("df shape:", df.shape)
    print("df cols num:", len(df.columns))
    print("df cols: \n", df.columns)
    print("df describe \n", df.describe())
    print("df not null ratio: \n", df.count()/len(df))
print_df(train_bank)

df shape: (10000, 39)
df cols num: 39
df cols: 
 Index(['loan_id', 'user_id', 'total_loan', 'year_of_loan', 'interest',
       'monthly_payment', 'class', 'employer_type', 'industry', 'work_year',
       'house_exist', 'censor_status', 'issue_date', 'use', 'post_code',
       'region', 'debt_loan_ratio', 'del_in_18month', 'scoring_low',
       'scoring_high', 'known_outstanding_loan', 'known_dero',
       'pub_dero_bankrup', 'recircle_b', 'recircle_u', 'initial_list_status',
       'app_type', 'earlies_credit_mon', 'title', 'policy_code', 'f0', 'f1',
       'f2', 'f3', 'f4', 'early_return', 'early_return_amount',
       'early_return_amount_3mon', 'is_default'],
      dtype='object')
df describe 
             loan_id        user_id    total_loan  year_of_loan      interest  \
count  1.000000e+04   10000.000000  10000.000000  10000.000000  10000.000000   
mean   1.025210e+06  225209.587700  14402.126591      3.479600     13.222782   
std    1.438682e+04   14386.820956   8953.946807     

In [3]:
train_bank.columns = ['loan_id', 'user_id', 'total_loan', 'year_of_loan', 'interest',
       'monthly_payment', 'class', 'employer_type', 'industry', 'work_year',
       'house_exist', 'censor_status', 'issue_date', 'use', 'post_code',
       'region', 'debt_loan_ratio', 'del_in_18month', 'scoring_low',
       'scoring_high', 'known_outstanding_loan', 'known_dero',
       'pub_dero_bankrup', 'recircle_b', 'recircle_u', 'initial_list_status',
       'app_type', 'earlies_credit_mon', 'title', 'policy_code', 'f0', 'f1',
       'f2', 'f3', 'f4', 'early_return', 'early_return_amount',
       'early_return_amount_3mon', 'is_default']

In [4]:
# ### 数据预处理

common_cols = []
for col in train_bank.columns:
    if col in train_internet.columns:
        common_cols.append(col)
    else: continue
len(common_cols)


print(len(train_bank.columns))
print(len(train_internet.columns))


train_bank_left = list(set(list(train_bank.columns)) - set(common_cols))
train_internet_left = list(set(list(train_internet.columns)) - set(common_cols))



train1_data = train_internet[common_cols]
train2_data = train_bank[common_cols]
test_data = test[common_cols[:-1]]


39
42


In [11]:
train1_data.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,...,policy_code,f0,f1,f2,f3,f4,early_return,early_return_amount,early_return_amount_3mon,is_default
0,119262,0,12000.0,5,11.53,264.10,B,普通企业,采矿业,NaN,...,1.0,1.0,0.0,8.0,17.0,8.0,0,0,0.0,1
1,369815,1,8000.0,3,13.98,273.35,C,普通企业,国际组织,10+ years,...,1.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0
2,787833,2,20000.0,5,17.99,507.76,D,上市企业,信息传输、软件和信息技术服务业,10+ years,...,1.0,6.0,0.0,10.0,8.0,3.0,0,0,0.0,0
3,671675,3,10700.0,3,10.16,346.07,B,普通企业,电力、热力生产供应业,2 years,...,1.0,3.0,0.0,4.0,11.0,6.0,0,0,0.0,0
4,245160,4,8000.0,3,8.24,251.58,B,政府机构,金融业,5 years,...,1.0,3.0,0.0,8.0,6.0,4.0,0,0,0.0,0


In [14]:
train1_data.groupby("class")["is_default"].sum() / train1_data.groupby("class")["is_default"].count()

class
A    0.060389
B    0.132806
C    0.224813
D    0.304684
E    0.384209
F    0.453950
G    0.497721
Name: is_default, dtype: float64

In [17]:
# In[8]:


import datetime

# 日期类型：issueDate，earliesCreditLine
# 转换为pandas中的日期类型
train1_data['issue_date'] = pd.to_datetime(train1_data['issue_date'])
# 提取多尺度特征
train1_data['issue_date_y'] = train1_data['issue_date'].dt.year
train1_data['issue_date_m'] = train1_data['issue_date'].dt.month
# 提取时间diff
# 设置初始的时间
base_time = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
# 转换为天为单位
train1_data['issue_date_diff'] = train1_data['issue_date'].apply(lambda x: x-base_time).dt.days
train1_data[['issue_date', 'issue_date_y', 'issue_date_m', 'issue_date_diff']]
train1_data.drop('issue_date', axis = 1, inplace = True)


# In[9]:


# 日期类型：issueDate，earliesCreditLine
# 转换为pandas中的日期类型
train2_data['issue_date'] = pd.to_datetime(train2_data['issue_date'])
# 提取多尺度特征
train2_data['issue_date_y'] = train2_data['issue_date'].dt.year
train2_data['issue_date_m'] = train2_data['issue_date'].dt.month
# 提取时间diff
# 设置初始的时间
base_time = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
# 转换为天为单位
train2_data['issue_date_diff'] = train2_data['issue_date'].apply(lambda x: x-base_time).dt.days
train2_data[['issue_date', 'issue_date_y', 'issue_date_m', 'issue_date_diff']]
train2_data.drop('issue_date', axis = 1, inplace = True)
train2_data


# In[10]:


employer_type = train1_data['employer_type'].value_counts().index
industry = train1_data['industry'].value_counts().index


# In[11]:


emp_type_dict = dict(zip(employer_type, [0,1,2,3,4,5]))
industry_dict = dict(zip(industry, [i for i in range(15)]))


# In[12]:
# 填充工作年限

train1_data['work_year'].fillna('10+ years', inplace=True)
train2_data['work_year'].fillna('10+ years', inplace=True)

work_year_map = {'10+ years': 10, '2 years': 2, '< 1 year': 0, '3 years': 3, '1 year': 1,
     '5 years': 5, '4 years': 4, '6 years': 6, '8 years': 8, '7 years': 7, '9 years': 9}
train1_data['work_year']  = train1_data['work_year'].map(work_year_map)
train2_data['work_year']  = train2_data['work_year'].map(work_year_map)

# 映射网络贷款等级

train1_data['class'] = train1_data['class'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6})
train2_data['class'] = train2_data['class'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6})

# 映射所在公司类型（世界五百强、国有企业、普通企业…）
train1_data['employer_type'] = train1_data['employer_type'].map(emp_type_dict)
train2_data['employer_type'] = train2_data['employer_type'].map(emp_type_dict)

# 工作领域（传统工业、商业、互联网、金融…）
train1_data['industry'] = train1_data['industry'].map(industry_dict)
train2_data['industry'] = train2_data['industry'].map(industry_dict)


# In[13]:


# 日期类型：issueDate，earliesCreditLine
#train[cat_features]
# 转换为pandas中的日期类型
test_data['issue_date'] = pd.to_datetime(test_data['issue_date'])
# 提取多尺度特征
test_data['issue_date_y'] = test_data['issue_date'].dt.year
test_data['issue_date_m'] = test_data['issue_date'].dt.month
# 提取时间diff
# 设置初始的时间
base_time = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
# 转换为天为单位
test_data['issue_date_diff'] = test_data['issue_date'].apply(lambda x: x-base_time).dt.days
test_data[['issue_date', 'issue_date_y', 'issue_date_m', 'issue_date_diff']]
test_data.drop('issue_date', axis = 1, inplace = True)
test_data['work_year'].fillna('10+ years', inplace=True)

work_year_map = {'10+ years': 10, '2 years': 2, '< 1 year': 0, '3 years': 3, '1 year': 1,
     '5 years': 5, '4 years': 4, '6 years': 6, '8 years': 8, '7 years': 7, '9 years': 9}
test_data['work_year']  = test_data['work_year'].map(work_year_map)
test_data['class'] = test_data['class'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6})
test_data['employer_type'] = test_data['employer_type'].map(emp_type_dict)
test_data['industry'] = test_data['industry'].map(industry_dict)


In [18]:
# ## 模型使用
# 1) LigthGBM
# 2) NN

# ##### 使用internet和bank数据共同特征总量训练

# In[16]:


import lightgbm
from sklearn import metrics

root_path = r"E:\project\20210928_loan"

X_train1 = train1_data.drop(['is_default','earlies_credit_mon','loan_id','user_id'], axis = 1, inplace = False)
y_train1 = train1_data['is_default']

X_train2 = train2_data.drop(['is_default','earlies_credit_mon','loan_id','user_id'], axis = 1, inplace = False)
y_train2 = train2_data['is_default']

X_train = pd.concat([X_train1, X_train2])
y_train = pd.concat([y_train1, y_train2])

X_test = test_data.drop(['earlies_credit_mon','loan_id','user_id'], axis = 1, inplace = False)

# 利用Internet数据预训练模型1
clf_ex=lightgbm.LGBMRegressor(n_estimators = 200)
clf_ex.fit(X = X_train, y = y_train)
clf_ex.booster_.save_model(root_path + r'\model\20210928_LGBMmodle.mdl')
pred = clf_ex.predict(X_test)


# In[17]:


# submission

submission = pd.DataFrame({'id':test['loan_id'], 'isDefault':pred})
submission.to_csv(root_path + r'\submit\submission.csv', index = None)

In [19]:
submission.dtypes

id              int64
is_default    float64
dtype: object

In [20]:
pred = clf_ex.predict(X_test)

# submission
pred = [0 if p < 0 else p for p in pred]
submission = pd.DataFrame({'id':test['loan_id'], 'isDefault':pred})
submission.to_csv(root_path + r'\submit\submission.csv', index = None)

In [5]:
common_cols

['loan_id',
 'user_id',
 'total_loan',
 'year_of_loan',
 'interest',
 'monthly_payment',
 'class',
 'employer_type',
 'industry',
 'work_year',
 'house_exist',
 'censor_status',
 'issue_date',
 'use',
 'post_code',
 'region',
 'debt_loan_ratio',
 'del_in_18month',
 'scoring_low',
 'scoring_high',
 'pub_dero_bankrup',
 'recircle_b',
 'recircle_u',
 'initial_list_status',
 'earlies_credit_mon',
 'title',
 'policy_code',
 'f0',
 'f1',
 'f2',
 'f3',
 'f4',
 'early_return',
 'early_return_amount',
 'early_return_amount_3mon',
 'is_default']